# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f64a819b820>
├── 'a' --> tensor([[ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059]])
└── 'x' --> <FastTreeValue 0x7f64a819baf0>
    └── 'c' --> tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                        [ 0.2735, -0.4188,  1.6806, -0.5757],
                        [ 1.6243,  0.4005, -0.8061, -0.4517]])

In [4]:
t.a

tensor([[ 1.0596, -1.2632, -0.3492],
        [-0.4961,  0.6181, -1.2059]])

In [5]:
%timeit t.a

61.6 ns ± 0.0818 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f64a819b820>
├── 'a' --> tensor([[ 0.7900,  0.7569,  0.0635],
│                   [ 1.3160, -0.0977,  0.6456]])
└── 'x' --> <FastTreeValue 0x7f64a819baf0>
    └── 'c' --> tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                        [ 0.2735, -0.4188,  1.6806, -0.5757],
                        [ 1.6243,  0.4005, -0.8061, -0.4517]])

In [7]:
%timeit t.a = new_value

67.6 ns ± 0.0137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0596, -1.2632, -0.3492],
               [-0.4961,  0.6181, -1.2059]]),
    x: Batch(
           c: tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0596, -1.2632, -0.3492],
        [-0.4961,  0.6181, -1.2059]])

In [11]:
%timeit b.a

53.8 ns ± 0.0327 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2445, -0.1665,  0.5538],
               [ 1.1855, -1.0121,  3.0432]]),
    x: Batch(
           c: tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.0738 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

865 ns ± 0.164 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 782 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f64a816aac0>
├── 'a' --> tensor([[[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]],
│           
│                   [[ 1.0596, -1.2632, -0.3492],
│                    [-0.4961,  0.6181, -1.2059]]])
└── 'x' --> <FastTreeValue 0x7f63fb56bcd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 84.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f63fae6df40>
├── 'a' --> tensor([[ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059],
│                   [ 1.0596, -1.2632, -0.3492],
│                   [-0.4961,  0.6181, -1.2059]])
└── 'x' --> <FastTreeValue 0x7f647c92eb50>
    └── 'c' --> tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                        [ 0.2735, -0.4188,  1.6806, -0.5757],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 36.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 89 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2250,  0.4055, -0.2865,  0.8737],
                       [ 0.2735, -0.4188,  1.6806, -0.5757],
                       [ 1.6243,  0.4005, -0.8061, -0.4517]],
              
                      [[-1.2250,  0.4055, -0.2865,  0.8737],
                       [ 0.2735, -0.4188,  1.6806, -0.5757],
                       [ 1.6243,  0.4005, -0.8061, -0.4517]],
              
                      [[-1.2250,  0.4055, -0.2865,  0.8737],
                       [ 0.2735, -0.4188,  1.6806, -0.5757],
                       [ 1.6243,  0.4005, -0.8061, -0.4517]],
              
                      [[-1.2250,  0.4055, -0.2865,  0.8737],
                       [ 0.2735, -0.4188,  1.6806, -0.5757],
                       [ 1.6243,  0.4005, -0.8061, -0.4517]],
              
                      [[-1.2250,  0.4055, -0.2865,  0.8737],
                       [ 0.2735, -0.4188,  1.6806, -0.5757],
                       [ 1.6243,  0.4005, -0.8061, -0.4517]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517],
                      [-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517],
                      [-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517],
                      [-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517],
                      [-1.2250,  0.4055, -0.2865,  0.8737],
                      [ 0.2735, -0.4188,  1.6806, -0.5757],
                      [ 1.6243,  0.4005, -0.8061, -0.4517],
                      [-1.2250,  0.4055, -0.2865,  0.8737],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
